In [1]:
import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import Point

In [2]:
crs = 'epsg:6372'
file_path = "../data/interim/00a_modified_test.shp"
data = pd.read_csv(file_path, index_col=0)
data.geometry = data.geometry.apply(loads)
data = gpd.GeoDataFrame(data, crs=crs)
data['centroid'] = data.geometry.apply(lambda x: x.centroid)
data['lon'] = data.centroid.apply(lambda point: point.xy[0][0])
data['lat'] = data.centroid.apply(lambda point: point.xy[1][0])
data['CVE_CONCAT'] = data.CVE_CONCAT.apply(lambda x: str(x).zfill(5))

In [3]:
center_CVE_CONCAT_point = gpd.GeoDataFrame(
    data.groupby('CVE_CONCAT').mean()[
        ['lon', 'lat']
    ].apply(
        lambda x: Point(x['lon'], x['lat']),
        axis=1
    ).reset_index().rename(
        columns={0: 'geometry'}
    ), crs=crs
)
center_CVE_CONCAT_point['lon_lat_geometry'] = center_CVE_CONCAT_point.geometry.to_crs(epsg=4326)
center_CVE_CONCAT_point['location_count'] = data.groupby('CVE_CONCAT').geometry.count().rename('location_count').reset_index()['location_count']

/mnt/c/Users/dhdzm/Documents/proyects/seguia/SeGuia/venv/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [135]:
#grouped_localities = data.dissolve('CVE_CONCAT')
#grouped_localities['centroid'] = grouped_localities.geometry.apply(lambda x: x.centroid)

In [4]:
file_path_mun = "../data/raw/mg_2020_integrado/conjunto_de_datos/00mun.shp"

data_mun = gpd.read_file(file_path_mun)


def list_of_intersecting_municipalities(geom):
    return list(data_mun[data_mun.geometry.apply(lambda x: geom.intersects(x))].CVEGEO)

data_mun['cve_concat_list_of_neighbour_municipalities'] = data_mun.geometry.apply(list_of_intersecting_municipalities)

def second_neighbours_interesecting_municipalities(neighbour_list):
    feature = 'cve_concat_list_of_neighbour_municipalities'
    neighbour_set = set(neighbour_list)
    second_neighbour_set = set(
        data_mun[data_mun.CVEGEO.isin(neighbour_list)][feature].sum()
    )
    return list(second_neighbour_set - neighbour_set)

data_mun['cve_concat_list_of_2nd_neighbour_municipalities'] = data_mun.cve_concat_list_of_neighbour_municipalities.apply(second_neighbours_interesecting_municipalities)

In [5]:
interesting_list = ['CVEGEO', 'NOMGEO', 'cve_concat_list_of_neighbour_municipalities', 'cve_concat_list_of_2nd_neighbour_municipalities', 'geometry']
new_data_mun = center_CVE_CONCAT_point.rename(columns={'geometry': 'point_geometry'}).merge(
    data_mun[interesting_list], 
    left_on='CVE_CONCAT', 
    right_on='CVEGEO',
)

In [6]:
new_data_mun[new_data_mun.NOMGEO =='Guadalajara']

,CVE_CONCAT,point_geometry,lon_lat_geometry,location_count,CVEGEO,NOMGEO,cve_concat_list_of_neighbour_municipalities,cve_concat_list_of_2nd_neighbour_municipalities,geometry
577,14039,POINT (2361383.557 963469.075),POINT (-103.33600 20.67248),443,14039,Guadalajara,"[14039, 14045, 14098, 14101, 14120, 14124]","[14029, 14005, 14070, 32028, 14105, 14009, 140...","POLYGON ((2363989.299 972099.751, 2364144.238 ..."


In [7]:
new_data_mun

,CVE_CONCAT,point_geometry,lon_lat_geometry,location_count,CVEGEO,NOMGEO,cve_concat_list_of_neighbour_municipalities,cve_concat_list_of_2nd_neighbour_municipalities,geometry
0,01001,POINT (2471409.903 1095700.338),POINT (-102.27803 21.87795),351,01001,Aguascalientes,"[01001, 01002, 01003, 01005, 01010, 01011, 140...","[14117, 01008, 01009, 32024, 32055, 11022, 320...","POLYGON ((2489072.503 1115771.584, 2489352.774..."
1,01002,POINT (2494330.383 1126438.693),POINT (-102.05525 22.15721),22,01002,Asientos,"[01001, 01002, 01006, 01009, 01010, 01011, 140...","[01008, 32025, 14035, 01007, 32038, 14116, 010...","POLYGON ((2494680.261 1141224.505, 2494749.948..."
2,01003,POINT (2427723.846 1095703.250),POINT (-102.70286 21.87662),26,01003,Calvillo,"[01001, 01003, 01005, 01008, 14116, 32018, 320...","[32048, 14025, 14035, 01007, 14081, 01010, 320...","POLYGON ((2429607.454 1120262.254, 2429071.902..."
3,01004,POINT (2469503.072 1148329.651),POINT (-102.29764 22.35566),10,01004,Cosío,"[01004, 01007, 32008, 32025]","[01008, 01009, 32024, 32036, 32035, 32012, 01006]","POLYGON ((2470517.824 1155028.588, 2470552.248..."
4,01005,POINT (2465053.591 1104883.875),POINT (-102.34006 21.96119),47,01005,Jesús María,"[01001, 01003, 01005, 01006, 01008, 01011]","[14035, 01009, 01007, 32055, 32018, 14116, 010...","POLYGON ((2465526.729 1114740.466, 2465752.546..."
...,...,...,...,...,...,...,...,...,...
2464,32054,POINT (2529349.827 1148982.076),POINT (-101.71354 22.36160),24,32054,Villa Hidalgo,"[24025, 32035, 32038, 32054]","[24015, 32025, 14064, 24021, 24049, 24022, 320...","POLYGON ((2531223.368 1166188.348, 2531242.386..."
2465,32055,POINT (2412175.141 1150919.431),POINT (-102.85728 22.37707),62,32055,Villanueva,"[01003, 01008, 14025, 14081, 32012, 32015, 320...","[32048, 32036, 01001, 32017, 32030, 32010, 320...","POLYGON ((2420011.778 1183552.394, 2420043.461..."
2466,32056,POINT (2439646.182 1193066.889),POINT (-102.59082 22.76096),184,32056,Zacatecas,"[32005, 32012, 32017, 32020, 32032, 32050, 320...","[32016, 01008, 14025, 24049, 01007, 32010, 320...","POLYGON ((2421707.751 1202019.971, 2421780.086..."
2467,32057,POINT (2462810.181 1190278.884),POINT (-102.36400 22.73635),67,32057,Trancoso,"[32016, 32017, 32036, 32057]","[32025, 24049, 32008, 32037, 32035, 32012, 320...","POLYGON ((2476976.320 1198274.003, 2476566.139..."


In [8]:
new_data_mun.to_csv("../data/interim/00mun_with_neighbours.csv")